In [61]:
import os, sys

sys.path.append('c:\\Projects\\taxi_predicts\\src\\')

In [62]:
import config as config

In [80]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

#we fetch raw data for the last 28days to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

print(fetch_data_from)

current_date=Timestamp('2024-03-20 13:00:00')
2024-02-21 13:00:00


In [108]:
#we cannot import the current data from data warehouse
# so we simulate the data

from data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    #we simulate production data by sampling historical data from 52weeks ago 

    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)

    print(from_date_)
    print(to_date_)

    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]
    
    rides = pd.concat([rides, rides_2])
    rides['pickup_datetime'] += timedelta(days=7*52)
    #shift the data to pretend this is recent data

    print(rides.head(10))
    print(rides_2.tail(10))

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [109]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

2023-02-22 13:00:00
2023-03-22 13:00:00
File 2023-02 was already in local storage
File 2023-03 was already in local storage
            pickup_datetime  pickup_location_id
1748114 2024-02-21 17:10:55                 229
1748115 2024-02-21 17:23:00                 263
1753080 2024-02-21 17:33:53                 263
1753081 2024-02-21 17:46:00                 263
1753082 2024-02-21 18:14:25                 230
1757220 2024-02-21 18:46:12                 237
1757221 2024-02-21 18:55:51                 229
1760434 2024-02-21 19:42:59                 141
1760435 2024-02-21 19:59:27                 263
1760436 2024-02-21 20:03:08                 262
            pickup_datetime  pickup_location_id
3376779 2023-03-22 12:01:10                  79
3376780 2023-03-22 12:32:00                  50
3376781 2023-03-22 12:51:00                 226
3376782 2023-03-22 12:05:25                 164
3376783 2023-03-22 12:57:00                 261
3376784 2023-03-22 12:58:51                 161
3376785 2023

In [96]:
from data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:01<00:00, 232.91it/s]


In [100]:
print(f'{ts_data.tail(10)}')

               pickup_hour  rides  pickup_location_id
178070 2024-03-20 03:00:00      2                 265
178071 2024-03-20 04:00:00      3                 265
178072 2024-03-20 05:00:00      5                 265
178073 2024-03-20 06:00:00      9                 265
178074 2024-03-20 07:00:00      3                 265
178075 2024-03-20 08:00:00      7                 265
178076 2024-03-20 09:00:00      2                 265
178077 2024-03-20 10:00:00      2                 265
178078 2024-03-20 11:00:00      4                 265
178079 2024-03-20 12:00:00      5                 265


In [90]:
import hopsworks
#now we connect to the feture_store and feature group to store the data

#connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

#connect to the feature store
feature_store = project.get_feature_store()

#connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description='Time-series data at hourly frequency',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
)



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012
Connected. Call `.close()` to terminate connection gracefully.


In [91]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 178080/178080 | Elapsed Time: 00:17 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549012/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2e3251a9ba0>, None)

In [ ]:
#now we need to automate this so it runs every hour, we do that in github actions